# Implementing an Physics-informed neural network for the 1D Heat equation using the PINN framework

In [ ]:
import sys
import numpy as np
import scipy.io
import torch
from pyDOE import lhs
from torch import Tensor, ones, stack, load
from torch.autograd import grad
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

### Underlying PDE

**PDE:** $ u_{t}-k u_{x x}=0 $ &emsp; $ 0 < x < 1 $

**IC:** $ u (x, 0) = f (x) $ &emsp; $ 0 < x < 1 $

**BC:** $ u (0,t) = u (1,t) = 0 $   &emsp; $ 0 < t < 1 $

$f:= u_{t}-k u_{x x} $  



In [ ]:
sys.path.append('...')  # PINNFramework etc.
import PINNFramework as pf

In [ ]:
#data
np.random.seed(0)
x = np.random.uniform(low = 0, high = 1, size = 256) 
t = np.sort(np.random.uniform(low = 0, high = 1, size = 201))
x = x.flatten()[:, None]
t = t.flatten()[:, None]

u = np.random.rand(x.shape[0],t.shape[0])

In [ ]:
class BoundaryConditionDataset(Dataset):

    def __init__(self, nb, lb, ub, t):
        """
        Constructor of the initial condition dataset

        Args:
          n0 (int)
        """
        super(type(self)).__init__()

        idx_t = np.random.choice(t.shape[0], nb, replace=False)
        tb = t[idx_t, :]
        self.x_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
        self.x_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.x_lb).float(), Tensor(self.x_ub).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

In [ ]:
class InitialConditionDataset(Dataset):

    def __init__(self, n0, x, t, u):
        """
        Constructor of the boundary condition dataset

        Args:
          n0 (int)
        """
        super(type(self)).__init__()
        
        Exact_u = u

        idx_x = np.random.choice(x.shape[0], n0, replace=False)
        self.x = x[idx_x, :]
        self.u = Exact_u[idx_x, 0:1]
        self.t = np.zeros(self.x.shape)


    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

    def __getitem__(self, idx):
        x = np.concatenate([self.x, self.t], axis=1)
        y = self.u
        return Tensor(x).float(), Tensor(y)

In [ ]:
class PDEDataset(Dataset):
    def __init__(self, nf, lb, ub):
        self.xf = lb + (ub - lb) * lhs(2, nf)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.xf).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

In [ ]:
if __name__ == "__main__":
    
    # Domain bounds
    lb = np.array([0, 0.0])
    ub = np.array([1, 1])

    # initial condition
    ic_dataset = InitialConditionDataset(n0=50, x=x, t=t, u=u)
    initial_condition = pf.InitialCondition(ic_dataset)
    
    # boundary conditions
    bc_dataset = BoundaryConditionDataset(nb=50, lb=lb, ub=ub, t=t)
    dirichlet_bc_u = pf.DirichletBC(u[0,:],bc_dataset, 0, "u dirichlet boundary condition")

    # PDE
    pde_dataset = PDEDataset(20000, lb, ub)


    def heatequation1d(x, u):
        pred = u
        u = pred[:, 0]
        print("x:", x.shape)
        grads = ones(u.shape, device=pred.device) # move to the same device as prediction
        grad_u = grad(u, x, create_graph=True, grad_outputs=grads)[0]

        # calculate first order derivatives
        u_x = grad_u[:, 0]
        print("u_x", u_x.shape)
        u_t = grad_u[:, 1]
        print("u_t", u_t.shape)

        # calculate second order derivatives
        grad_u_x = grad(u_x, x, create_graph=True, grad_outputs=grads)[0]
        
        u_xx = grad_u_x[:, 0]
        print("u_xx", u_xx.shape)

        # thermal diffusivity k = 0.5
        f_u = u_t - 0.5 * u_xx
        print("f_u.shape", f_u.shape)

        return f_u


    pde_loss = pf.PDELoss(pde_dataset, heatequation1d)
    model = pf.models.MLP(input_size=2, output_size=2, hidden_size=100, num_hidden=4, lb=lb, ub=ub)
    pinn = pf.PINN(model, 2, 2, pde_loss, initial_condition, [dirichlet_bc_u], use_gpu=False)
    pinn.fit(100, 'Adam', 1e-3)
    
    